In [69]:
import warnings
warnings.filterwarnings("ignore")

import os
import shutil

import torch
import torchaudio
from torch import nn
!pip install torchmetrics
import torchvision.models as models

from torchmetrics import Accuracy,Precision, Recall, F1Score,FBetaScore
from torch.optim import Adam, AdamW, Adagrad,RMSprop,SGD

!pip install torchvision
import torchvision.transforms as transform

from torch.cuda.amp import autocast

!pip install wget
!pip install timm==0.4.5

import wget
import timm

from timm.models.layers import to_2tuple,trunc_normal_

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader


from tqdm import tqdm
import numpy as np

from IPython.display import FileLink


%matplotlib inline
%load_ext tensorboard

import librosa 
import torch.nn.functional as Fx
from torch.autograd import Variable
import sys
from collections import OrderedDict

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/ESC-50

In [71]:
# пути и адреса
PATHS = {
    'to_audio':'../input/environmental-sound-classification-50/audio/audio/44100/', # путь к папке с аудиофайлами
    'to_dataset': '../input/environmental-sound-classification-50/esc50.csv', # путь к .csv файлу
    'to_tensors' : './tensors/', # Путь к папке, в которую сохраняются тензоры данных аудиофайлов
    'to_original_tensors': './tensors/original/', # папка с оригинальными данными
    'to_normalized_tensors': './tensors/normalized/', # папка с стандартизованными данными
    'to_params': './params/',# папка с параметрами данных и модели
    'to_z_score_params': './params/z_score', # папка с средними и дисперсиями, посчитанными попеременно по всем тренировочным fold
    
    'to_pretrained_weights_url':'https://www.dropbox.com/s/cv4knew8mvbrnvq/audioset_0.4593.pth?dl=1', # ссылка на веса предобученной модели AST
    'to_pretrained_model_weights':  './params/pretrained_model_weights.pth', # путь, по которому сохранятся указанные выше веса
    
    'to_trained_model_weights_url': './params/best_model_weights.pth', #'https://drive.google.com/file/d/1uSAeMslIhqJHK5k8cdOsdabRx6grSEmZ/view?usp=sharing',# ccылка на лучшие веса модели, достигнутые в процессе обучения
    'to_trained_model_weights': './params/current_model_weights.pth',# путь, по которому сохранятся указанные выше веса
    
    'to_pretrained_cnn_weights': '../input/mypkl/mx-h64-1024_0d3-1.17.pkl'
}

# ниже установлены рекоммендуемые параметры для обучения модели
CONFIG = {
    # метки loss, metric, optimizer, scheduler имеют набор допустимых значений. Устанновив такое значение, вы создадите соответствующий объект внутри модели
    'seed': 42,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'model': 'ast',#'ast' 'cnn', 
    'num_classes': 50,
    
    'metric': 'accuracy', # возможные значения:  accuracy, f1, recall, precision
    'average': 'micro', #  возможные значения: micro, macro
    'loss': 'logloss', # возможные значения: logloss
    'epoches': 300,#2
    'lr': 8e-6, #8e-6,
    
    'train_batch_size': 2,#2 # ВНИМАНИЕ! Не рекомендуется увеличивать значение этого параметра, если ваш GPU < 13GB. Используемае здесь модель занимает очень много места в памяти и обучать на больших батчах при небольшом GPU не получится.
    'test_batch_size':80, #4,
    
    'feature_size': (1025, 216), # размерность представлений спектрограмм, с которыми будет работать модель
    
    'type_of_load_weights': 'pretrained',# веса, которые нужно загрузить
    # pretrained - веса оригинальной модели AST, trained - лучшие веса модели, которые удалось получить за все время обучения
    
    # параметры распределения для z-стандартизации
    'type_of_distribution_params': 'original', # допустимые значения: original, current. original - вычисленные на AudioSet для модели AST, current - вычисленные на текущем датасете
    'original_mean': -4.2677393,
    'original_std': 4.5689974,


    #optimizer############
    'optimizer': 'adam',# возможные значения: adam, adamw, sgd, rmsprop, adagrad
    
    'weight_decay': 0, # параметр регуляризации
    'momentum': 1e-1, # моментум (только для SGD и RMSprop)
    'smooth': 0.99,
    'eps': 1e-8,# значение для численной стабильности знаменателя (только для RMSprop)
    
    # модификатор шага оптимайзера
    #scheduler ###########
    'scheduler': 'cosine', # возможные значения:  cosine, exponential, step
    'lr_decay_factor': 0.96, # коэффициент, на который на каждом шаге умножается lr (только для exponential)
    'step_size' : 200, # период снижения скорости обучения (только для step)
    'min_lr': 1e-6, # # минимально возможный lr (только для cosine)
    'max_iter': 2400#4800, # максимум итераций для работы алгоритма (только для cosine)
}



# Предусмотрена возможность добавления в модель пользовательских loss, metric, scheduler и optimizer
# Формат добавления на примере метрики: {'metric': ('название метрики', класс_метрики, [список параметров в порядке их следования в объявлении метрики])}
# Обратите внимение, что пользовательские инструменты "перекрывают" те, которые были объявлены в CONFIG

CUSTOM_TOOLS = {
    #'loss': ('mlsmloss', torch.nn.MultiLabelSoftMarginLoss, []),
    #'metric': ('fbeta', FBetaScore, [CONFIG['num_classes'],0.5, 0.95,'macro']),
    #'optimizer': ('radam', torch.optim.RAdam, [0.001,(0.9, 0.999),1e-08, 0]),
    #'scheduler': ('lambdalr', torch.optim.lr_scheduler.LambdaLR, [lambda epoch: epoch // 30])
}

DATASET_CONFIG = {
    'tfr': 'fbank'# Частотно-временные представления. Допустимые значения : mfcc, fbank
}

# создание необходимых папок
if not os.path.exists(PATHS['to_tensors']):
    os.mkdir(PATHS['to_tensors'])
    os.mkdir(PATHS['to_original_tensors'])
    os.mkdir(PATHS['to_normalized_tensors'])
if not os.path.exists(PATHS['to_params']):
    os.mkdir(PATHS['to_params'])
    
def set_seed(seed=42):
    '''
    Фиксирование сида
    '''
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(CONFIG['seed'])

Изучим датасет. 

In [72]:
features = pd.read_csv(PATHS['to_dataset'])
features

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A


Из всех столбцов, полезными для обучения модели являются только 3: filename, fold, target.

filename - это путь к аудиофайлу.

Покажем, что каждый filename в датасете уникален:

In [73]:
print('Всего записей в датасете: ', features.shape[0])
print('Уникальных имен файлов: ', len(features.filename.unique()))

Всего записей в датасете:  2000
Уникальных имен файлов:  2000


 Из этого файла при помощи библиотеки torchaudio следует извлечь данные mel-спектрограммы.

In [74]:
filename = features.loc[1,:].filename 
waveform, sr = torchaudio.load(PATHS['to_audio'] + filename)

feature_bank = torchaudio.compliance.kaldi.fbank(
    waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
    window_type='hanning', num_mel_bins=216, dither=0.0,
    frame_shift=10)

feature_bank

tensor([[-12.5984, -11.8655, -15.9424,  ..., -14.0511, -13.2287, -12.6782],
        [-11.1301, -10.3245, -14.0157,  ..., -13.4380, -13.0887, -12.9269],
        [-15.9424, -15.9424, -15.9424,  ..., -13.2381, -13.5167, -13.3581],
        ...,
        [-12.9755, -12.3088, -15.9424,  ..., -12.6964, -13.1629, -12.7698],
        [-15.9424, -15.7064, -15.9424,  ..., -13.3250, -12.8344, -12.9705],
        [-15.9424, -15.0021, -15.9424,  ..., -13.4089, -13.1005, -13.9848]])

target - номер метки класса, которому сопоставляется аудиофайл.

In [75]:
features.target.value_counts()

0     40
39    40
29    40
10    40
7     40
26    40
6     40
44    40
23    40
20    40
49    40
24    40
28    40
14    40
18    40
2     40
35    40
38    40
21    40
1     40
11    40
42    40
5     40
33    40
40    40
12    40
43    40
27    40
36    40
19    40
30    40
34    40
9     40
22    40
48    40
41    40
47    40
31    40
17    40
45    40
8     40
15    40
46    40
37    40
32    40
16    40
25    40
4     40
3     40
13    40
Name: target, dtype: int64

Видно, что каждого класса в датасете одинаковое количество. Значит, датасет сбалансирован.

fold - папки, по которым разбит датасет для удобства обучения.

In [76]:
res = []
for i in range(features.fold.max()):
  ds_ = features[features.fold == i+1]
  res.append(ds_.target.value_counts())
  
pd.DataFrame(res)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
target,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
target,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
target,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
target,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
target,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


Видно, что в каждой папке находится одинаковое количество объектов каждого класса.

На основе описанных выше наблюдений можно сделать вывод, что в качестве главной метрики для модели-классификатора можно выбрать accuracy.

In [77]:
class PreprocessorESC50:
    '''
    Предобработчик данных. Удаляет лишние данные. Сохраняет данные в виде тензоров, вычисляет параметры распределения по тренировочным папкам 
    '''
    def __init__(self,CONFIG, DATASET_CONFIG, PATHS):
        self.__path_to_dataset = PATHS['to_dataset']
        self.__path_to_audio =  PATHS['to_audio']
        self.__path_to_orig_tensors = PATHS['to_original_tensors']
        self.__path_to_norm_tensors = PATHS['to_normalized_tensors']
        self.__path_to_params = PATHS['to_params']
        self.__path_to_z_score_params = PATHS['to_z_score_params']
        
        self.__feature_size = CONFIG['feature_size']
        
        self.features  = pd.read_csv(self.__path_to_dataset)
        self.target = self.features['target']
        
        self.tfr = DATASET_CONFIG['tfr']
        
    def __create_feature_bank(self, wav_name, mel_bins, target_length):
        '''
        Извлекает из аудиофрагмента набор данных,полученных с помощью mel-спектрограммы
        
        wav_name - имя файла
        mel_bins - количество частотных ячеек
        target_length - размерность выходных данных
        '''
        waveform, sr = torchaudio.load(wav_name)

        feature_bank = torchaudio.compliance.kaldi.fbank(
            waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
            window_type='hanning', num_mel_bins=mel_bins, dither=0.0,
            frame_shift=10)

        n_frames = feature_bank.shape[0]

        difference = target_length - n_frames
        if difference > 0:# Если до размерности выходных данных немного не хватает, то дозаполним данные нулями
            pad = torch.nn.ZeroPad2d((0, 0, 0, difference)) 
            feature_bank = pad(feature_bank)
        elif difference < 0: # Если данных слишком много - обрежем их
            feature_bank = feature_bank[0:target_length, :]

        return feature_bank
    
    def __create_mfcc(self, wav_name, num_seps, target_length):
        waveform, sr = torchaudio.load(wav_name)
        feature = torchaudio.compliance.kaldi.mfcc(waveform, htk_compat=True,num_ceps=num_seps, sample_frequency=sr, use_energy=False,
            window_type='hanning', num_mel_bins=num_seps, dither=0.0,
            frame_shift=10)
        n_frames = feature.shape[0]

        difference = target_length - n_frames
        if difference > 0:# Если до размерности выходных данных немного не хватает, то дозаполним данные нулями
            pad = torch.nn.ZeroPad2d((0, 0, 0, difference)) 
            feature = pad(feature)
        elif difference < 0: # Если данных слишком много - обрежем их
            feature = feature[0:target_length, :]
        return feature
        
    def __create_and_save_tensors(self, row):
        '''
        Загружает файл, извлекает и сохраняет признаки и целевые переменные в виде тензоров
        row - запись в датасете
        '''
        filename = row['filename']
        idx = row.name 
        path = ''.join([self.__path_to_audio, filename]) # место сохранения тензора
        
        if self.tfr == 'fbank':
            feature = self.__create_feature_bank(path, self.__feature_size[1], self.__feature_size[0])
        elif self.tfr == 'mfcc':
            feature = self.__create_mfcc(path, self.__feature_size[1], self.__feature_size[0])
        

        path_to_feature = ''.join([self.__path_to_orig_tensors, filename]) 
        path_to_target = ''.join([self.__path_to_orig_tensors, filename, '_target']) 
        features_tensor = torch.Tensor(feature)

        target_tensor = torch.Tensor(self.target[idx])

        torch.save(features_tensor, path_to_feature)
        torch.save(target_tensor, path_to_target)
        return path_to_feature
    
    def __compute_z_score_params(self):
        '''
        Метод подсчета параметров распределения для z-стандартизации
        '''
        max_num_fold = self.features['fold'].max()
        counts = np.zeros((max_num_fold,))
        means = np.zeros((max_num_fold,))
        stds = np.zeros((max_num_fold,))

        for i, data in self.features.iterrows():
            tensor = torch.load(''.join([self.__path_to_orig_tensors, data.filename])) 
            mean = torch.mean(tensor) # среднее по тензору
            std = torch.std(tensor) # стандартное отклонение по тензору
            for j in range(0, max_num_fold): # Считаем параметры распределения по всем комбинациям из 4 тренировочных папок
                if j + 1 != data.fold:
                    stds[j] += std
                    means[j] += mean
                    counts[j] += 1
        means /= counts 
        stds /= counts  

        self.__z_params = np.concatenate([means.reshape(1, -1), stds.reshape(1, -1)], axis=0)
        torch.save(torch.Tensor(self.__z_params), self.__path_to_z_score_params) # сохраняем параметры распределения

    def preprocess(self):
        '''
        Основной метод предобработки данных
        '''
        self.ohe = OneHotEncoder(sparse=False)
        self.target = self.ohe.fit_transform(self.target.to_numpy().reshape(-1, 1)) # представляем target в виде one-hot векторов
        _ = self.features.apply(self.__create_and_save_tensors, axis=1) # создаем и сохраняем тензоры
        self.features.drop(labels = ['target', 'category', 'esc10', 'src_file', 'take'], axis=1, inplace=True) # удаляем ненужные данные
        self.__compute_z_score_params() # вычисляем и сохраняем параметры распределения
        return self.features, self.target

In [78]:
dataset = PreprocessorESC50(CONFIG,DATASET_CONFIG, PATHS)
dataset.preprocess()  # ВНИМАНИЕ! Данные метод стоит выполнять только при первом запуске ноутбука. Все результаты его работы будут сохранены, и сам метод
# можно будет закомментировать. Это существенно повысит скорость выполнения ноутбука

(               filename  fold
 0      1-100032-A-0.wav     1
 1     1-100038-A-14.wav     1
 2     1-100210-A-36.wav     1
 3     1-100210-B-36.wav     1
 4     1-101296-A-19.wav     1
 ...                 ...   ...
 1995   5-263831-B-6.wav     5
 1996  5-263902-A-36.wav     5
 1997   5-51149-A-25.wav     5
 1998    5-61635-A-8.wav     5
 1999     5-9032-A-0.wav     5
 
 [2000 rows x 2 columns],
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]))

Выведем параметры распределения по каждой папке:

In [79]:
params = torch.load(PATHS['to_z_score_params'])
d = {'mean': params[0], 'std': params[1]}
pd.DataFrame(data=d, index = np.arange(1, 6))

,mean,std
1,-3.234985,4.445635
2,-3.240178,4.444564
3,-3.208234,4.438404
4,-3.238236,4.453884
5,-3.238322,4.461284


Как можно заметить, распределение данных по всем папкам приблизительно одинаковое.

In [80]:
class DatasetECS50:
    '''
    Класс предобработанного датасета, с возможностью итерации по нему.
    Реализованы z-стандартизации,
    а также метод разбиения на тренировочный и тестовый датасет.
    '''
    def __init__(self, PATHS, features, target):
        self.__path_to_dataset = PATHS['to_dataset']
        self.__path_to_audio =  PATHS['to_audio']
        self.__path_to_orig_tensors = PATHS['to_original_tensors']
        self.__path_to_norm_tensors = PATHS['to_normalized_tensors']
        self.__path_to_params = PATHS['to_params']
        self.__path_to_z_score_params = PATHS['to_z_score_params']
        self.features = features
        self.target = target

        self.type_of_dist_params = CONFIG['type_of_distribution_params']
        self.original_mean = CONFIG['original_mean']
        self.original_std = CONFIG['original_std']

    def __len__(self):
        '''
        Метод, необходимый для возможности итерирования по датасету
        '''
        return self.features.shape[0]
    
    def train_test_split(self, test_fold_num):
        '''
        Разбиение датасетов на тестовый и тренировочный по номеру папки. Возвращает эти самые датасеты
        '''
        self.cure_fold = test_fold_num # фиксируем тренировочную папку
        train = self.features[self.features.fold != test_fold_num].reset_index() # выбираем все данные, соответствующие тренировочным папкам
        test = self.features[self.features.fold == test_fold_num].reset_index() # выбираем данные, соответствующие тестовой папке
        train['filename'].map(self.__normalize_and_save) # стандартизуем, для более быстрой сходимости алгоритма 
        test['filename'].map(self.__normalize_and_save)
        
        train_index = train.set_index('index').index
        test_index = test.set_index('index').index
        train.drop(labels = ['index'], axis=1, inplace=True)
        test.drop(labels = ['index'], axis=1, inplace=True)
        
        return DatasetECS50(PATHS, train, self.target[train_index]), DatasetECS50(PATHS, test, self.target[test_index])

    def __apply_z_score(self, filename):   
        '''
        Метод применения z-стандартизации к тензору
        '''
        tensor = torch.load(''.join([self.__path_to_orig_tensors, filename]))
        
        fold_of_cur_record = self.cure_fold

        if self.type_of_dist_params == 'original': # стандартизация с помощью параметров распределения модели на датасете AudioSet, на котором она предобучалась 
            tensor = (tensor.view(1, tensor.shape[0], tensor.shape[1]) - (self.original_mean)) / (self.original_std * 2)
        elif self.type_of_dist_params == 'current':
            z_score_params = torch.load(self.__path_to_z_score_params) # стандартизация с помащью параметров распределения этого датасета
            tfm = transform.Normalize(mean=z_score_params[0][fold_of_cur_record - 1], std=z_score_params[1][fold_of_cur_record - 1])
            tensor = tfm(tensor.view(1, tensor.shape[0], tensor.shape[1]))
        return tensor
 
    def __normalize_and_save(self, filename):
        '''
        Метод нормализации и сохранения тензоров
        '''
        tensor = self.__apply_z_score(filename)
        path_to_feature = ''.join([self.__path_to_norm_tensors, filename])
        torch.save(tensor, path_to_feature)
        
    def __getitem__(self, ids):
        '''
        Метод, необходимый для возможности итерирования по датасету
        Возвращает пару (данные, целевая переменная)
        '''
        filename = self.features.loc[ids,:].filename
        tensor = torch.load(''.join([self.__path_to_norm_tensors, filename]))
        tensor = tensor.view(tensor.shape[1], tensor.shape[2])
        target_tensor = torch.load(''.join([self.__path_to_orig_tensors, filename, '_target']))
        
        return (tensor, target_tensor) 

In [81]:
ds = DatasetECS50(PATHS , dataset.features, dataset.target)
train, test = ds.train_test_split(test_fold_num=5)
train.features

,filename,fold
0,1-100032-A-0.wav,1
1,1-100038-A-14.wav,1
2,1-100210-A-36.wav,1
3,1-100210-B-36.wav,1
4,1-101296-A-19.wav,1
...,...,...
1595,4-99193-B-4.wav,4
1596,4-99644-A-4.wav,4
1597,4-99644-B-4.wav,4
1598,4-99644-C-4.wav,4


В процессе выбора модели, я пришел к выводу, что данная задача плохо решается с помощью обычных RNN и CNN, а действительно хороших результатов можно достичь только с помощью архитектур Transformer или подобных им.

Именно поэтому свой выбор модели я остановил на Audio Spectrogram Transformer (AST), предобученный на датасетах ImageNet и AudioSet, взятый практически без изменений из открытого доступа. Данная модель показыват одни из самых лучших результатов при работе с аудиофрагментами. 

Эта модель практически не имеет недостатков. Из серьезных минусов можно выделить только то, что она довольно тяжеловесная, и если загружать ее стандартного размера GPU Kaggle или Colab, то придется обучать модель небольшими батчами данных, т.к. большую часть памяти GPU будет занимать именно модель.

Код AST я практически без изменений оставил в ячейке ниже.

In [82]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()

        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class ASTModel(nn.Module):
    '''
    Audio Spectrogram Transformer, предобученный на датасетах ImageNet и AudioSet, взятый практически без изменений из открытого доступа.
    '''
    def __init__(self,PATHS, label_dim=527, fstride=10, tstride=10, input_fdim=128, input_tdim=1024, imagenet_pretrain=True, audioset_pretrain=False, model_size='base384', verbose=False):
        
        super(ASTModel, self).__init__()
        assert timm.__version__ == '0.4.5', 'Please use timm == 0.4.5, the code might not be compatible with newer versions.'
        
        if verbose == True:
            print('---------------AST Model Summary---------------')
            print('ImageNet pretraining: {:s}, AudioSet pretraining: {:s}'.format(str(imagenet_pretrain),str(audioset_pretrain)))
        timm.models.vision_transformer.PatchEmbed = PatchEmbed

        if not audioset_pretrain:
            if model_size == 'tiny224':
                self.v = timm.create_model('vit_deit_tiny_distilled_patch16_224', pretrained=imagenet_pretrain)
            elif model_size == 'small224':
                self.v = timm.create_model('vit_deit_small_distilled_patch16_224', pretrained=imagenet_pretrain)
            elif model_size == 'base224':
                self.v = timm.create_model('vit_deit_base_distilled_patch16_224', pretrained=imagenet_pretrain)
            elif model_size == 'base384':
                self.v = timm.create_model('vit_deit_base_distilled_patch16_384', pretrained=imagenet_pretrain)
            else:
                raise Exception('Model size must be one of tiny224, small224, base224, base384.')
                
            self.original_num_patches = self.v.patch_embed.num_patches
            self.oringal_hw = int(self.original_num_patches ** 0.5) 
            self.original_embedding_dim = self.v.pos_embed.shape[2]
            self.mlp_head = nn.Sequential(nn.LayerNorm(self.original_embedding_dim), nn.Linear(self.original_embedding_dim, label_dim))

            f_dim, t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim)
            num_patches = f_dim * t_dim
            self.v.patch_embed.num_patches = num_patches
            if verbose == True:
                print('frequncey stride={:d}, time stride={:d}'.format(fstride, tstride))
                print('number of patches={:d}'.format(num_patches))

            new_proj = torch.nn.Conv2d(1, self.original_embedding_dim, kernel_size=(16, 16), stride=(fstride, tstride))
            if imagenet_pretrain == True:
                new_proj.weight = torch.nn.Parameter(torch.sum(self.v.patch_embed.proj.weight, dim=1).unsqueeze(1))
                new_proj.bias = self.v.patch_embed.proj.bias
            self.v.patch_embed.proj = new_proj

            if imagenet_pretrain == True:
                new_pos_embed = self.v.pos_embed[:, 2:, :].detach().reshape(1, self.original_num_patches, self.original_embedding_dim).transpose(1, 2).reshape(1, self.original_embedding_dim, self.oringal_hw, self.oringal_hw)
                if t_dim <= self.oringal_hw:
                    new_pos_embed = new_pos_embed[:, :, :, int(self.oringal_hw / 2) - int(t_dim / 2): int(self.oringal_hw / 2) - int(t_dim / 2) + t_dim]
                else:
                    new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(self.oringal_hw, t_dim), mode='bilinear')
                if f_dim <= self.oringal_hw:
                    new_pos_embed = new_pos_embed[:, :, int(self.oringal_hw / 2) - int(f_dim / 2): int(self.oringal_hw / 2) - int(f_dim / 2) + f_dim, :]
                else:
                    new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(f_dim, t_dim), mode='bilinear')
                new_pos_embed = new_pos_embed.reshape(1, self.original_embedding_dim, num_patches).transpose(1,2)
                self.v.pos_embed = nn.Parameter(torch.cat([self.v.pos_embed[:, :2, :].detach(), new_pos_embed], dim=1))
            else:
                new_pos_embed = nn.Parameter(torch.zeros(1, self.v.patch_embed.num_patches + 2, self.original_embedding_dim))
                self.v.pos_embed = new_pos_embed
                trunc_normal_(self.v.pos_embed, std=.02)

        elif audioset_pretrain:
            if audioset_pretrain and not imagenet_pretrain:
                raise ValueError('currently model pretrained on only audioset is not supported, please set imagenet_pretrain = True to use audioset pretrained model.')
            if model_size != 'base384':
                raise ValueError('currently only has base384 AudioSet pretrained model.')
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            self.audio_model = ASTModel(PATHS, label_dim=527, fstride=10, tstride=10, input_fdim=128, input_tdim=1024, imagenet_pretrain=False, audioset_pretrain=False, model_size='base384', verbose=False)
            self.audio_model = torch.nn.DataParallel(self.audio_model)
            self.pretrained_weights = PATHS['to_pretrained_model_weights']
            self.pretrained_weights_url = PATHS['to_pretrained_weights_url']
            if not os.path.exists(self.pretrained_weights):
                wget.download(self.pretrained_weights_url, out=self.pretrained_weights)
            self.set_pretrained_weights()
            self.v = self.audio_model.module.v
            self.original_embedding_dim = self.v.pos_embed.shape[2]
            self.mlp_head = nn.Sequential(nn.LayerNorm(self.original_embedding_dim), nn.Linear(self.original_embedding_dim, label_dim))

            f_dim, t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim)
            num_patches = f_dim * t_dim
            self.v.patch_embed.num_patches = num_patches
            if verbose:
                print('frequncey stride={:d}, time stride={:d}'.format(fstride, tstride))
                print('number of patches={:d}'.format(num_patches))

            new_pos_embed = self.v.pos_embed[:, 2:, :].detach().reshape(1, 1212, 768).transpose(1, 2).reshape(1, 768, 12, 101)
            if t_dim < 101:
                new_pos_embed = new_pos_embed[:, :, :, 50 - int(t_dim/2): 50 - int(t_dim/2) + t_dim]
            else:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(12, t_dim), mode='bilinear')
            if f_dim < 12:
                new_pos_embed = new_pos_embed[:, :, 6 - int(f_dim/2): 6 - int(f_dim/2) + f_dim, :]
            elif f_dim > 12:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(f_dim, t_dim), mode='bilinear')
            new_pos_embed = new_pos_embed.reshape(1, 768, num_patches).transpose(1, 2)
            self.v.pos_embed = nn.Parameter(torch.cat([self.v.pos_embed[:, :2, :].detach(), new_pos_embed], dim=1))

    def get_shape(self, fstride, tstride, input_fdim=128, input_tdim=1024):
        test_input = torch.randn(1, 1, input_fdim, input_tdim)
        test_proj = nn.Conv2d(1, self.original_embedding_dim, kernel_size=(16, 16), stride=(fstride, tstride))
        test_out = test_proj(test_input)
        f_dim = test_out.shape[2]
        t_dim = test_out.shape[3]
        return f_dim, t_dim

    def set_pretrained_weights(self):
        state_dict = torch.load(self.pretrained_weights, map_location=self.device)
        self.audio_model.load_state_dict(state_dict, strict=False)
    

    @autocast()
    def forward(self, x):
        x = x.unsqueeze(1)
        x = x.transpose(2, 3)

        B = x.shape[0]
        x = self.v.patch_embed(x)
        cls_tokens = self.v.cls_token.expand(B, -1, -1)
        dist_token = self.v.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)
        for blk in self.v.blocks:
            x = blk(x)
        x = self.v.norm(x)
        x = (x[:, 0] + x[:, 1]) / 2

        x = self.mlp_head(x)
        return x

In [83]:
class CNN(nn.Module):
    def __init__(self, nclass):
        super(CNN,self).__init__() 
        self.globalpool = Fx.avg_pool2d
        self.layer1 = nn.Sequential(nn.Conv2d(1,16,kernel_size=3,padding=1),nn.BatchNorm2d(16),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(16,16,kernel_size=3,padding=1),nn.BatchNorm2d(16),nn.ReLU())
        self.layer3 = nn.MaxPool2d(2)

        self.layer4 = nn.Sequential(nn.Conv2d(16,32,kernel_size=3,padding=1),nn.BatchNorm2d(32),nn.ReLU())
        self.layer5 = nn.Sequential(nn.Conv2d(32,32,kernel_size=3,padding=1),nn.BatchNorm2d(32),nn.ReLU())
        self.layer6 = nn.MaxPool2d(2)

        self.layer7 = nn.Sequential(nn.Conv2d(32,64,kernel_size=3,padding=1),nn.BatchNorm2d(64),nn.ReLU())
        self.layer8 = nn.Sequential(nn.Conv2d(64,64,kernel_size=3,padding=1),nn.BatchNorm2d(64),nn.ReLU())
        self.layer9 = nn.MaxPool2d(2)

        self.layer10 = nn.Sequential(nn.Conv2d(64,128,kernel_size=3,padding=1),nn.BatchNorm2d(128),nn.ReLU())
        self.layer11 = nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1),nn.BatchNorm2d(128),nn.ReLU())
        self.layer12 = nn.MaxPool2d(2)

        self.layer13 = nn.Sequential(nn.Conv2d(128,256,kernel_size=3,padding=1),nn.BatchNorm2d(256),nn.ReLU())
        self.layer14 = nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1),nn.BatchNorm2d(256),nn.ReLU())
        self.layer15 = nn.MaxPool2d(2) #

        self.layer16 = nn.Sequential(nn.Conv2d(256,512,kernel_size=3,padding=1),nn.BatchNorm2d(512),nn.ReLU())
        self.layer17 = nn.MaxPool2d(2) # 
        
        self.layer18 = nn.Sequential(nn.Conv2d(512,1024,kernel_size=2),nn.BatchNorm2d(1024),nn.ReLU())
        self.layer19 = nn.Sequential(nn.Conv2d(1024,nclass,kernel_size=1))

    def forward(self,x):
        x = x.reshape(x.shape[0], -1, x.shape[1],x.shape[2])
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out = self.layer18(out)
        out1 = self.layer19(out) 
        out = self.globalpool(out1,kernel_size=out1.size()[2:])
        out = out.view(out.size(0),-1)
        return out #,out1

In [84]:
class AudioClassifier:
    '''
    Модель для задачи классификации датасета ESC-50. Имеет возможность удобной настройки параметров через CONFIG. Для изменения объектов обучения - 
    loss, optimizer, scheduler, metric - достаточно просто подать правильную метку в CONFIG.
    Реализованы методы обычного обучения и кросс-валидации. 

    На каждом этапе валидации, при достижении более высокого результата метрики, веса модели сохраняются. Их можно скачать с помощью метода download weights
    '''
    def __init__(self, CONFIG, PATHS, CUSTOM_TOOLS):
        ##### общие характеристики #####
        self.device = CONFIG['device']
        self.num_classes = CONFIG['num_classes']
        self.model_name = CONFIG['model']
        self.lr = CONFIG['lr']
        self.epoches = CONFIG['epoches']
        self.train_batch_size = CONFIG['train_batch_size']
        self.test_batch_size = CONFIG['test_batch_size']
        self.path_to_trained_weights = PATHS['to_trained_model_weights'] # путь к лучшим весам этой модели, которые удалось получить за все время обучения
        self.__feature_size = CONFIG['feature_size'] # требуемая размерность WxH данных
        
        self.type_of_load_weights = CONFIG['type_of_load_weights']
        
        self.path_to_pretrained_cnn_weights = PATHS['to_pretrained_cnn_weights']
        
        ##### модель #####
        if self.model_name == 'ast':
            self.model = ASTModel(
                PATHS,
                input_tdim =self.__feature_size[0],
                input_fdim=self.__feature_size[1],
                label_dim=self.num_classes,
                audioset_pretrain=True,
            ).to(self.device)
            # веса для AST при значении параметра type_of_load_weights == 'pretrained' скачиваются при инициализации AST
            # при значении trained - скачиваются наилучшие веса, полученные в процессе обучения
            if self.type_of_load_weights == 'trained':
                self.path_to_trained_weights_url = PATHS['to_trained_model_weights_url']
                if not os.path.exists(self.path_to_trained_weights): # если по указанному адресу ничего нет - скачиваем туда веса классификатора.
                    wget.download(self.path_to_trained_weights_url, out=self.path_to_trained_weights)                                                                    
                self.load_weights(self.path_to_trained_weights)
        elif self.model_name == 'cnn':
            self.model = CNN(self.num_classes).to(self.device)
            self.load_weights(self.path_to_pretrained_cnn_weights, model_name='cnn')

        self.softmax = nn.Softmax(dim=1)

        ##### оптимизаторы #####
        if CUSTOM_TOOLS.get('optimizer') is not None: # если пользователь подал кастомный оптимизатор, то выбираем именно его, иначе тот, который указана в конфиге
            self.optimizer_name = CUSTOM_TOOLS['optimizer'][0]
            self.optimizer = CUSTOM_TOOLS['optimizer'][1](self.model.parameters(), *CUSTOM_TOOLS['optimizer'][2])
        else:
            self.optimizer_name = CONFIG['optimizer']
            self.weight_decay = CONFIG['weight_decay']
            
            if self.optimizer_name == 'adamw':
                self.optimizer = AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
            elif self.optimizer_name == 'adam':
                self.optimizer = Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
            elif self.optimizer_name == 'sgd':
                self.momentum = CONFIG['momentum']
                self.optimizer = SGD(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay,momentum=self.momentum) 
            elif self.optimizer_name == 'adagrad':
                self.optimizer = Adagrad(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
            elif self.optimizer_name == 'rmsprop':
                self.momentum = CONFIG['momentum']
                self.smooth = CONFIG['smooth']
                self.eps = CONFIG['smooth']
                self.optimizer = RMSprop(
                    self.model.parameters(),
                    lr=self.lr,
                    alpha=self.smooth,
                    eps=self.eps,
                    weight_decay=self.weight_decay, 
                    momentum=self.momentum
                )
            self.type_of_optimizers = {'adamw': AdamW, 'adam': Adam, 'sgd': SGD, 'adagrad': Adagrad, 'rmsprop':RMSprop}

        ##### метрики #####

        if CUSTOM_TOOLS.get('metric') is not None:# если пользователь подал кастомную метрику, то выбираем именно ее, иначе ту, который указана в конфиге
            self.metric_name = CUSTOM_TOOLS['metric'][0]
            self.metric = CUSTOM_TOOLS['metric'][1](*CUSTOM_TOOLS['metric'][2])
        else:
            self.metric_name = CONFIG['metric']
            self.metric_average = CONFIG['average']

            if self.metric_name == 'accuracy':
                self.metric = Accuracy(average=self.metric_average, num_classes=self.num_classes)
            elif self.metric_name == 'f1':
                self.metric = F1Score(average=self.metric_average,num_classes=self.num_classes) 
            elif self.metric_name == 'precision':
                self.metric = Precision(average=self.metric_average,num_classes=self.num_classes)
            elif self.metric_name == 'recall':
                self.metric = Recall(average=self.metric_average,num_classes=self.num_classes)
            
            
        ##### функции потерь #####
        if CUSTOM_TOOLS.get('loss') is not None: # если пользователь подал кастомный лосс, то выбираем именно его, иначе тот, который указана в конфиге
            self.criterion_name = CUSTOM_TOOLS['loss'][0]
            self.criterion = CUSTOM_TOOLS['loss'][1](*CUSTOM_TOOLS['loss'][2]).to(self.device)
        else:
            self.criterion_name = CONFIG['loss']
            if self.criterion_name == 'logloss':
                self.criterion = nn.CrossEntropyLoss().to(self.device) 

        
        
        ##### модификаторы для шага #####
        if CUSTOM_TOOLS.get('scheduler') is not None: # если пользователь подал кастомный scheduler, то выбираем именно его, иначе тот, который указана в конфиге
            self.scheduler_name = CUSTOM_TOOLS['scheduler'][0]
            self.scheduler = CUSTOM_TOOLS['scheduler'][1](self.optimizer, *CUSTOM_TOOLS['scheduler'][2])
        else:
            self.max_iter = CONFIG['max_iter']
            self.scheduler_name = CONFIG['scheduler']
            self.min_lr = CONFIG['min_lr']

            if self.scheduler_name == 'cosine':
                self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer,T_max=self.max_iter,  eta_min=self.min_lr, verbose=False)
            elif self.scheduler_name == 'exponential':
                self.lr_decay_factor = CONFIG['lr_decay_factor']
                self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=self.lr_decay_factor, verbose=False)
            elif self.scheduler_name == 'step':
                self.lr_decay_factor = CONFIG['lr_decay_factor']
                self.step_size = CONFIG['step_size']
                self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=self.step_size, gamma=self.lr_decay_factor,verbose=False)
            
    def save_weights(self, path='./start_weigths'):
        '''
        Метод сохранения весов модели по указанному адресу
        '''
        if os.path.exists(path):
            os.remove(path)
        torch.save(self.model.state_dict(), path)

    def load_weights(self, path='./start_weigths', model_name='ast'):
        '''
        Метод сохранения весов модели по указанному адресу
        '''
        if model_name == 'ast':
            state_dict = torch.load(path, map_location=self.device)
            self.model.load_state_dict(state_dict, strict=False)
        elif model_name == 'cnn':
            #self.path1 = 'https://github.com/anuragkr90/weak_feature_extractor/blob/master/mx-h64-1024_0d3-1.17.pkl'
            self.path_to_pretrained_weights = '../input/mypkl/mx-h64-1024_0d3-1.17.pkl'
            #self.path_to_pretrained_weights = './params/pretrained_cnn_weights.pkl'
            #if not os.path.exists(self.path_to_pretrained_weights):
            #    wget.download(self.path1, out=self.path_to_pretrained_weights) 

            state_dict = torch.load(path, map_location=self.device)
            #print(state_dict)
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():

                if 'module.' in k:
                    name = k[7:]
                else:
                    name = k
                if k in ['layer19.0.weight', 'layer19.0.bias']:
                    new_state_dict[name] = v
            self.model.load_state_dict(new_state_dict, strict=False)

    def download_weights(self):
        '''
        Метод для скачивания весов модели
        '''
        return FileLink(self.path_to_trained_weights)
    
    
    def train(self, train_dataloader):
        '''
        Метод для обучения. Отображает оценки функции потерь и метрики в бар консоли.
        train_dataloader - подгрузчик тренировочных данных по батчам
        '''
        self.model.train()
        sum_loss = 0.
        sum_metric_scores= 0.
        cnt = 0
        bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for _, (x, y) in bar:
            x = x.to(self.device) # перебрасываем данные на GPU
            y = y.to(self.device)

            pred = self.model(x) 
            loss = self.criterion(pred, y) 

            loss.backward() # считаем градиенты
            
            self.optimizer.step() # делаем шаг в направлении минимума
            self.optimizer.zero_grad()
            self.scheduler.step() # уменьшаем lr
            
            pred = self.softmax(pred) # получаем вероятности
            pred_ = pred.detach().to('cpu') # переносим данные из GPU на CPU
            
            y_ =  torch.argmax(y.detach().to('cpu').int(), dim=1) # считаем метрику
            metric = self.metric(pred_, y_).item()

            if cnt % 202 == 0: # "обнуление" счетчика и переменных сумм для отображения более актуальных средних значений loss и metric
                cnt=1
                sum_loss = loss.item()
                sum_metric_scores = metric
            else: # во всех иных случаях, высчитываем средние значения метрики
                cnt+=1
                sum_loss += loss.item()
                sum_metric_scores += metric

            bar.set_postfix(mean_train_loss=sum_loss/cnt, mean_train_metric=sum_metric_scores/cnt) # отображаем актуальные значения в баре
            
    def val(self, test_dataloader):
        '''
        Метод для валидации.Отображает оценки функции потерь и метрики в бар консоли.
        train_dataloader - подгрузчик тестовых данных по батчам.
        '''
        bar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
        cnt = 0
        sum_loss = 0.
        sum_metric_scores = 0.
        cnt = 0
        self.model.eval()
        with torch.no_grad(): # отключаем вычисление градиента
            for _, (x, y) in bar: 
                x = x.to(self.device)
                y = y.to(self.device)
                
                pred = self.model(x) 
                loss = self.criterion(pred, y)

                pred = self.softmax(pred) 

                pred_ = pred.detach().to('cpu')
                y_ =  torch.argmax(y.detach().to('cpu').int(), dim=1)

                metric = self.metric(pred_, y_).item()

                cnt+=1
                sum_loss += loss.item()
                sum_metric_scores += metric
                bar.set_postfix(mean_valid_loss=sum_loss/cnt, mean_valid_metric=sum_metric_scores/cnt)  
        return sum_metric_scores/cnt
    
    def fit(self, train, test, save_best_weights=False):
        '''
        Метод обучения. Вохвращает последний результат метрики на валидации.
        Если модель бьет рекорд по метрике - сохраняет ее веса.
        train - тренировочные данные
        test - тестовые данные.
        '''
        train_dataloader = torch.utils.data.DataLoader( # тренировочный подгрузчик данных
            train,
            batch_size=self.train_batch_size ,
            shuffle=True,# перемешиваем данные
            pin_memory=True,
            drop_last=True
        ) 
        
        test_dataloader = torch.utils.data.DataLoader( # тестовый подгрузчик
            test,
            batch_size=self.test_batch_size,
            shuffle=True,
            pin_memory=True,
            drop_last=True
        )
        self.best_score = 0.
        for i in range(self.epoches):
            self.train(train_dataloader)
            res = self.val(test_dataloader)
            if res > self.best_score:
                self.best_score = res
                if save_best_weights:
                    self.save_weights(self.path_to_trained_weights) 
        return res
    
    def cross_validation_fit(self, dataset):
        '''
        Метод кросс-валидации. Выводит в консоль средний результат метрики по кросс-валидации
        dataset - полный набор данных
        '''
        ds = DatasetECS50(PATHS , dataset.features, dataset.target)
        max_num_fold = dataset.features['fold'].max()
        self.save_weights() # сохраняем изначальные веса
        result = 0.
        for f in range(max_num_fold):
            self.load_weights() # на каждом новом разбиении папок подгружаем изначальные веса
            self.optimizer = self.type_of_optimizers[self.optimizer_name](self.model.parameters(), lr=self.lr)
            train, test = ds.train_test_split(test_fold_num=f+1)
            result += self.fit(train, test)
        print('mean_{0}: '.format(self.metric_name), result/max_num_fold)

In [ ]:
clf = AudioClassifier(CONFIG, PATHS, CUSTOM_TOOLS)
clf.fit(train, test, save_best_weights=True)

Лучшие оценки, которых мне удалось достичь:

Метрика - accuracy

train - 1.0

valid - 0.925